# Beers - Solved

In [ ]:
# Import findpsark
import findspark

# Configure the environment
findspark.init()

# Import the Spark components required for the context creation
from pyspark import SparkConf, SparkContext

# Configure and create the context
conf = SparkConf()
conf = conf.setAppName('mds-session')
conf = conf.setMaster('local[*]')
sc = SparkContext.getOrCreate(conf=conf)

In [ ]:
# Leemos los ficheros de entrada
beers = sc.textFile('./data/beers.csv')
breweries = sc.textFile('./data/breweries.csv')

# Procesamos los RDD leídos para darles estructura
def parseBeers(line):
    return line.split(',')

def parseBreweries(line):
    return line.split(',')

beers = beers.map(parseBeers)
breweries = breweries.map(parseBreweries)

In [ ]:
# 1. ¿Cuántas filas hay en el set de datos de cervezas? ¿Y en el de 
# cervecerías?

beers.count()
breweries.count()

In [ ]:
# 2. ¿Cuál es el nombre de la cerveza que tiene más alcohol? 
# Devolver sólo un registro y sólo con el nombre.

def parseAlcohol(beer):
    alcohol = float(beer[1]) if beer[1] != '' else 0.0
    return (beer[4], alcohol)

max_alcohol = beers.map(parseAlcohol).sortBy(lambda beer: beer[1], ascending = False)
max_alcohol.map(lambda beer: beer[0]).first()

In [ ]:
# 3. ¿Cuántas cervezas sin alcohol (con nivel de alcohol a 0) hay en el 
# set de datos?

def parseAlcohol(beer):
    alcohol = float(beer[1]) if beer[1] != '' else 0.0
    return (alcohol)

beers.map(parseAlcohol).filter(lambda beer: beer == 0).count()

In [ ]:
# 4. ¿Cuál es el peso medio calculado sobre todas las cervezas?

def parseOunces(beer):
    ounces = float(beer[7]) if beer[7] != '' else 0.0
    return (ounces)

beers.map(parseOunces).mean()

In [ ]:
# 5. ¿Cuál es el el máximo nivel de alcohol por tipo de cerveza? 
# Ordena los resultados por nivel de alcohol de mayor a menor.

def parseStyleAlcohol(beer):
    style = beer[5]
    alcohol = float(beer[1]) if beer[1] != '' else 0.0
    return (style, alcohol)

group_style_alcohol = beers.map(parseStyleAlcohol)
group_style_alcohol = group_style_alcohol.reduceByKey(max)
group_style_alcohol = group_style_alcohol.sortBy(lambda style: style[1], ascending = False)
group_style_alcohol.collect()

In [ ]:
# 6. ¿Cuál es el nombre de la cervecería con la cerveza menos amarga? 
# Devolver sólo un registro y sólo con el nombre.

def parseBreweryBitter(beer):
    brewery_id = beer[6]
    bitter = float(beer[2]) if beer[2] != '' else -999
    return (brewery_id, bitter)

def parseBreweryName(brewery):
    brewery_id = brewery[0]
    name = brewery[1]
    return (brewery_id, name)

brewery_bitter = beers.map(parseBreweryBitter).filter(lambda beer: beer[1] >= 0.0)
brewery_name = breweries.map(parseBreweryName)
joined = brewery_bitter.join(brewery_name).map(lambda join: join[1])
min_bitter = joined.sortBy(lambda brewery: brewery[0])
min_bitter.map(lambda beer: beer[1]).first()

In [ ]:
# 7. ¿Cuál es la cerveza más amarga de Chicago? Devolver sólo un registro 
# y sólo con el nombre.

def parseBreweryBitter(beer):
    brewery_id = beer[6]
    name = beer[4]
    bitter = float(beer[2]) if beer[2] != '' else 0.0
    return (brewery_id, (name, bitter))

def parseBreweryCity(brewery):
    brewery_id = brewery[0]
    city = brewery[2]
    return (brewery_id, city)

brewery_bitter = beers.map(parseBreweryBitter)
brewery_city = breweries.map(parseBreweryCity)
joined = brewery_bitter.join(brewery_city).map(lambda join: join[1])
chicago_beers = joined.filter(lambda beer: beer[1] == 'Chicago').map(lambda beer: beer[0])
max_bitter = chicago_beers.sortBy(lambda beer: beer[1], ascending = False)
max_bitter.map(lambda beer: beer[0]).first()

In [ ]:
# 8 .¿Cuál es el estado que tiene más cervezas y cuántas tiene? Devolver sólo 
# un registro y sólo con las siglas.

def parseBreweryCount(beer):
    brewery_id = beer[6]
    return (brewery_id, 1)

def parseBreweryState(brewery):
    brewery_id = brewery[0]
    state = brewery[3]
    return (brewery_id, state)

brewery_count = beers.map(parseBreweryCount).reduceByKey(lambda el1, el2: el1 + el2)
brewery_state = breweries.map(parseBreweryState)
joined = brewery_count.join(brewery_state).map(lambda join: (join[1][1], join[1][0]))
max_state = joined.reduceByKey(lambda el1, el2: el1 + el2).sortBy(lambda state: state[1], ascending = False)
max_state.first()

In [ ]:
# 9. ¿Cuál es la desviación estándar de nivel de alcohol, amargor y peso por 
# ciudad y tipo de cerveza? Ordena los resultados por ciudad y tipo de 
# cerveza ascendente.

def parseBeerStyleAlcoholBitterOunces(beer):
    brewery_id = beer[6]
    style = beer[5]
    alcohol = float(beer[1]) if beer[1] != '' else 0.0
    bitter = float(beer[2]) if beer[2] != '' else 0.0
    ounces = float(beer[7]) if beer[7] != '' else 0.0
    return (brewery_id, (style, alcohol, bitter, ounces))

def parseBreweryCity(brewery):
    brewery_id = brewery[0]
    city = brewery[2]
    return (brewery_id, city)

import numpy as np
def numpyStd(element):
    cityStyle = element[0]
    alcohol = [data[2] for data in element[1]]
    bitter = [data[3] for data in element[1]]
    ounces = [data[4] for data in element[1]]
    return (cityStyle, np.std(np.array(alcohol)), np.std(np.array(bitter)), np.std(np.array(ounces)))

beer_info = beers.map(parseBeerStyleAlcoholBitterOunces)
brewery_city = breweries.map(parseBreweryCity)
joined = beer_info.join(brewery_city).map(lambda join: (join[1][1], join[1][0][0], join[1][0][1], join[1][0][2], join[1][0][3]))
city_style_info = joined.groupBy(lambda cityStyle: (cityStyle[0], cityStyle[1])).map(numpyStd)
city_style_info.sortBy(lambda cityStyle: cityStyle[0]).collect()

In [ ]:
# 10. ¿Qué ciudad vende la cerveza con el nombre más largo? Devuelve sólo un 
# registro con el nombre de la ciudad, el nombre de la cerveza y la longitud 
# del nombre.

def parseBreweryNameLength(beer):
    brewery_id = beer[6]
    name = beer[4]
    name_length = len(beer[4])
    return (brewery_id, (name, name_length))

def parseBreweryCity(brewery):
    brewery_id = brewery[0]
    city = brewery[2]
    return (brewery_id, city)

brewery_name_length = beers.map(parseBreweryNameLength)
brewery_city = breweries.map(parseBreweryCity)
joined = brewery_name_length.join(brewery_city).map(lambda join: join[1])
max_length = joined.sortBy(lambda beer: beer[0][1], ascending = False)
max_length.map(lambda beer: (beer[1], beer[0][0], beer[0][1])).first()

In [ ]:
# Stop the spark context
sc.stop()